# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
from sklearn.model_selection import GridSearchCV

In [12]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("wikipedia_train3.csv", parse_dates=True, encoding='UTF-8')
test = pd.read_csv("wikipedia_test3.csv", parse_dates=True, encoding='UTF-8')

## Step 1. EDA and Data Preprocessing

In [13]:
%%time
train['date'] = train['date'].apply(pd.to_datetime)

Wall time: 3min 2s


In [14]:
test['date'] = test['date'].apply(pd.to_datetime)

Checking data for NaN values...

In [15]:
train.isnull().sum()

Page      0
date      0
Visits    0
dtype: int64

In [16]:
test.isnull().sum()

Page      0
date      0
Visits    0
dtype: int64

In [7]:
pp.ProfileReport(train)

Number of variables,3
Number of observations,2882860
Total Missing (%),0.0%
Total size in memory,66.0 MiB
Average record size in memory,24.0 B
Numeric,1
Categorical,1
Date,1
Text (Unique),0
Rejected,0
Distinct count,11815


In [8]:
pp.ProfileReport(test)

Number of variables,3
Number of observations,732530
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,24.0 B
Numeric,1
Categorical,1
Date,1
Text (Unique),0
Rejected,0
Distinct count,11815


In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
#encoding categorical features
enc_page = LabelEncoder()
enc_page.fit(train.Page.values)
train['page_id'] = enc_page.transform(train.Page.values)
test['page_id'] = enc_page.transform(test.Page.values)

In [19]:
def get_month(d):
    return d.month
def get_day(d):
    return d.day
def get_weekend(d):
    if d.dayofweek > 4:
        return 1
    else:
        return 0
def get_dayofweek(d):
    return d.dayofweek

In [20]:
#creating new features
train['month'] = train.date.apply(get_month)
test['month'] = test.date.apply(get_month)
train['day'] = train.date.apply(get_day)
test['day'] = test.date.apply(get_day)
train['weekend'] = train.date.apply(get_weekend)
test['weekend'] = test.date.apply(get_weekend)

In [21]:
train['dayofweek'] = train.date.apply(get_dayofweek)
test['dayofweek'] = test.date.apply(get_dayofweek)

In [22]:
train.head()

,Page,date,Visits,page_id,month,day,weekend,dayofweek
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0,8212,1,1,0,4
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0,9801,1,1,0,4
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0,7151,1,1,0,4
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0,8247,1,1,0,4
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0,1594,1,1,0,4


In [23]:
#SMAPE
import math
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

## Step 2. Last 15 training days median

In [24]:
#there is a 9 days gap between train and test
print('Last train date: {}\nFirst test date:  {}'.format(
    train.date.values[-1],
    test.date.values[0]
))

Last train date: 2016-08-31T00:00:00.000000000
First test date:  2016-09-10T00:00:00.000000000


In [25]:
print('Days in test: {}'.format(
    len(test.date.value_counts())
))

Days in test: 62


In [26]:
#taking last 2 months to holdout set
start_date = pd.to_datetime('2016-01-01')
def one_split_validation(df):
    train_ = df[(df.date >= start_date) & (df.date < (start_date + pd.offsets.relativedelta(months=6)))]
    cv_ = df[df.date >= (start_date + pd.offsets.relativedelta(months=6))]
    return train_, cv_

In [27]:
new_train, cv_ = one_split_validation(train)

In [28]:
lastdaytrain = new_train.date.values[-1]

In [29]:
cv_.head()

,Page,date,Visits,page_id,month,day,weekend,dayofweek
2150330,Special:Search_zh.wikipedia.org_all-access_spider,2016-07-01,765.0,8212,7,1,0,4
2150331,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-07-01,6206.0,9801,7,1,0,4
2150332,Project:上传_zh.wikipedia.org_all-access_spider,2016-07-01,4253.0,7151,7,1,0,4
2150333,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-07-01,382.0,8247,7,1,0,4
2150334,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-07-01,16318.0,1594,7,1,0,4


In [30]:
new_p = pd.to_datetime(lastdaytrain) - pd.offsets.relativedelta(days=14)

In [31]:
visits_pred = new_train[new_train.date >= new_p].groupby('Page').median().reset_index()
visits_pred.rename(columns={"Visits": "pred_Visits"}, inplace=True)
print('CV score: {}'.format(
    smape_fast(cv_.Visits, cv_.merge(visits_pred, on=["Page"], how='left').pred_Visits)
))
print('Test score: {}'.format(
    smape_fast(test.Visits, test.merge(visits_pred, on=["Page"], how='left').pred_Visits)
))

CV score: 41.016810453819595
Test score: 49.961388431904616


## Step 3. K-fold

In [32]:
train_dates = ['2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01']
cv_dates = ['2016-03-10', '2016-04-10', '2016-05-10', '2016-06-10']
dates = tuple(zip(train_dates, cv_dates))

In [33]:
def create_validation(df, dates):
    train_date = pd.to_datetime(dates[0])
    cv_date = pd.to_datetime(dates[1])
    train_idx = df[(df.date >= (train_date - pd.offsets.relativedelta(months=2))) & \
                  (df.date < train_date)].index
    cv_idx = df[(df.date < (cv_date + pd.offsets.relativedelta(months=2))) & \
               (df.date >= cv_date)].index
    return train_idx, cv_idx

In [34]:
CV_iter = [create_validation(train, date) for date in dates]

In [35]:
from sklearn.tree import DecisionTreeRegressor

In [36]:
model_traffic = DecisionTreeRegressor(
    random_state=42
)

In [37]:
features_traffic = ['page_id', 'month', 'day', 'weekend', 'dayofweek']

In [38]:
params_traffic = {
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}
gs_traffic = GridSearchCV(
    estimator=model_traffic,
    param_grid=params_traffic,
    cv=CV_iter,
    scoring='mean_absolute_error',
    n_jobs=3,
    verbose=2
)
gs_traffic.fit(train[features_traffic], train['Visits'])

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  1.3min
[Parallel(n_jobs=3)]: Done  40 out of  40 | elapsed:  1.5min finished


GridSearchCV(cv=[(Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            708890, 708891, 708892, 708893, 708894, 708895, 708896, 708897,
            708898, 708899],
           dtype='int64', length=708900), Int64Index([ 815235,  815236...2622925, 2622926,
            2622927, 2622928, 2622929],
           dtype='int64', length=720715))],
       error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best'),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='mean_absolute_erro

In [39]:
gs_traffic.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=50, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [40]:
#test score
smape_fast(test.Visits, gs_traffic.best_estimator_.predict(test[features_traffic]))

44.9048931786664

In [41]:
cv_scores = []
for train_idx, cv_idx in CV_iter:
    best_tree = DecisionTreeRegressor(max_depth=50)
    best_tree.fit(train.loc[train_idx, features_traffic], train.loc[train_idx, 'Visits'])
    preds = best_tree.predict(train.loc[cv_idx, features_traffic])
    score = smape_fast(train.loc[cv_idx, 'Visits'], preds)
    cv_scores.append(score)
    print(score)
print('Mean CV score: {}'.format(np.mean(cv_scores)))

48.1853294067302
47.87504902523726
47.70579826218052
50.29368408302572
Mean CV score: 48.514965194293424


In [47]:
for i, pair in enumerate(CV_iter):
    print('fold {} summary \n'.format(i+1), train.loc[pair[1], 'Visits'].describe(), '\n')

fold 1 summary 
 count    7.207150e+05
mean     1.093453e+04
std      2.015142e+05
min      0.000000e+00
25%      2.123000e+03
50%      3.554000e+03
75%      6.187000e+03
max      2.018113e+07
Name: Visits, dtype: float64 

fold 2 summary 
 count    7.207150e+05
mean     1.095429e+04
std      2.164309e+05
min      0.000000e+00
25%      2.096000e+03
50%      3.491000e+03
75%      6.025000e+03
max      2.188997e+07
Name: Visits, dtype: float64 

fold 3 summary 
 count    7.207150e+05
mean     1.086375e+04
std      2.261315e+05
min      0.000000e+00
25%      1.984000e+03
50%      3.315000e+03
75%      5.828000e+03
max      2.320938e+07
Name: Visits, dtype: float64 

fold 4 summary 
 count    7.207150e+05
mean     1.377927e+04
std      4.796216e+05
min      0.000000e+00
25%      1.839000e+03
50%      3.094000e+03
75%      5.573000e+03
max      6.726426e+07
Name: Visits, dtype: float64 



In [48]:
test.Visits.describe()

count    7.325300e+05
mean     1.129125e+04
std      2.910576e+05
min      0.000000e+00
25%      1.607000e+03
50%      2.977000e+03
75%      5.321000e+03
max      3.344698e+07
Name: Visits, dtype: float64

As seen above, all folds are pretty similiar. But if to compare each of them with test, especially 4th fold, which result is worse than others (~ 50), standard deviation of last fold is much bigger than test's. 

In [51]:
from scipy.stats import ttest_ind
for i, _ in enumerate(CV_iter):
    print('T-test result: {}, fold {}'.format(ttest_ind(
        test.Visits, train.loc[CV_iter[i][1]].Visits, equal_var=False
).pvalue, i+1))

T-test result: 0.38969589873578037, fold 1
T-test result: 0.4278786991865795, fold 2
T-test result: 0.3223378347542646, fold 3
T-test result: 0.0001612557504920709, fold 4


T-test proved that last fold and test data are statistically different. (pvalue < 0.05)

We cannot omit the fact that the maximum value for the first 3 samples is lower than in the test
AND
perecentiles also vary a bit.

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [2]:
train_fr = pd.read_csv('train_french.csv')
test_fr = pd.read_csv('test_french.csv')

In [4]:
pp.ProfileReport(train_fr)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,23
Categorical,6
Date,0
Text (Unique),0
Rejected,0
Distinct count,83


In [5]:
pp.ProfileReport(test_fr)

Number of variables,29
Number of observations,75968
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,232.0 B
Numeric,23
Categorical,6
Date,0
Text (Unique),0
Rejected,0
Distinct count,83


As seen above, 'var4' seems to be redundant as it doesn't vary a lot аlong data.

In [3]:
categorical = [
    'nature_lri',
    'niv_urbanisation',
    'sit_fam',
    'var9',
    'veranda'
]

from sklearn.preprocessing import LabelEncoder
for col in categorical:
    enc = LabelEncoder()
    enc.fit(train_fr[col])
    train_fr[col] = enc.transform(train_fr[col])
    test_fr[col] = enc.transform(test_fr[col])

In [9]:
features_fr = train_fr.columns.drop(['prime_tot_ttc', 'var4']).values

In [14]:
#all values in 'profession' column
professions = train_fr.profession.value_counts().index.values

In [29]:
#all professions
new_prof = []
for elem in professions:
    if ';' in elem:
        for e in elem.split(';'):
            new_prof.append(e.replace(' ', ''))
    else:
        new_prof.append(elem)

In [36]:
prof_df_train = pd.DataFrame(data=np.zeros((train_fr.shape[0], len(new_prof))), columns=new_prof)

In [49]:
#creating additional features with professions (train)
for p in new_prof:
    for i, line in enumerate(train_fr.profession):
        if p in line:
            prof_df_train.loc[i, p] = 1

In [52]:
prof_df_test = pd.DataFrame(data=np.zeros((test_fr.shape[0], len(new_prof))), columns=new_prof)

In [53]:
#creating additional features with professions (test)
for p in new_prof:
    for i, line in enumerate(test_fr.profession):
        if p in line:
            prof_df_test.loc[i, p] = 1

In [56]:
train_fr = pd.concat([train_fr, prof_df_train], axis=1)
test_fr = pd.concat([test_fr, prof_df_test], axis=1)

In [73]:
train_fr.drop('profession', 1, inplace=True)
test_fr.drop('profession', 1, inplace=True)

In [11]:
features_fr = train_fr.columns.drop(['prime_tot_ttc', 'var4']).values #update features

In [50]:
from scipy.stats import normaltest
def find_cv(train_, test_):
    indices = train_.sample(n=1000, replace=False).index
    cv = train_[indices]
    stddev_test = test_.describe()['std']
    stddev_cv = cv.describe()['std']
    if (normaltest(cv).statistic > 1e-3) & (normaltest(test_).statistic > 1e-3):
        if stddev_test != stddev_cv:
            ttest_res = ttest_ind(cv, test_, equal_var=False).pvalue
        else:
            ttest_res = ttest_ind(cv, test_, equal_var = True).pvalue
    return indices, train_.drop(indices, 0).index, ttest_res

In [86]:
#taking 5 samples to validate on them
cv_iter = []
for i in range(5):
    best_t = 0
    while best_t < 0.06:
        cv_idx, train_idx, best_t = find_cv(train_fr['prime_tot_ttc'], test_fr['prime_tot_ttc'])
    cv_iter.append((train_idx, cv_idx))

In [87]:
from sklearn.ensemble import GradientBoostingRegressor

In [88]:
model = GradientBoostingRegressor(
    random_state=42
)

In [89]:
params_fr = {
    'max_depth': [10]
}
gs_fr = GridSearchCV(
    estimator=model,
    param_grid=params_fr,
    cv=cv_iter,
    scoring='mean_absolute_error',
    n_jobs=3,
    verbose=2
)
if __name__ == '__main__':
    gs_fr.fit(train_fr[features_fr], train_fr['prime_tot_ttc'])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Done   3 out of   5 | elapsed: 29.6min remaining: 19.8min
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed: 49.7min finished


In [90]:
gs_fr.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=1.0, verbose=0, warm_start=False)

In [97]:
cv_scores_fr = []
for train_idx, cv_idx in cv_iter:
    best_booster = GradientBoostingRegressor(
        max_depth=10,
        random_state=42
    )
    best_booster.fit(train_fr.loc[train_idx, features_fr], train_fr.loc[train_idx, 'prime_tot_ttc'])
    preds = best_booster.predict(train_fr.loc[cv_idx, features_fr])
    score = mape_(train_fr.loc[cv_idx, 'prime_tot_ttc'], preds)
    cv_scores_fr.append(score)
    print(score)
print('Mean CV score: {}'.format(np.mean(cv_scores_fr)))

8.799673333732573
9.10980839909588
8.573203071716154
8.364971992037402
8.167104170305315
Mean CV score: 8.602952193377465


In [91]:
best_grad = gs_fr.best_estimator_

In [92]:
preds = best_grad.predict(test_fr[features_fr])

In [95]:
def mape_(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [96]:
#test score
mape_(test_fr['prime_tot_ttc'].values, preds)

9.057822280184325